In [ ]:
import numpy as np

def sphere (x) :
    return np.sum (np.square(x), axis=1)

def rastrigin (x) :
    A = 10
    return A*x.shape[1] + np.sum(np.square(x) - A*np.cos(2*np.pi*x) , axis=1)

In [ ]:
def logisticMap (x, r) :
    return r * x * (1 - x) 

def tentMap (x, mu) :
    if x <= mu :
        return x/mu
    else :
        return (1-x)/(1-mu)

def lorenzFlow (initCond, param, t_end, length) :
    from scipy.integrate import odeint
    def lorenz (X, t, sigma, beta, rho) :
        x, y, z = X
        dXdt = [sigma*(y - x), x*(rho - z) - y, x*y - beta*z]
        return dXdt

    X0 = initCond
    t = np.linspace (0, t_end, length)
    return odeint (lorenz, X0, t, args = param)

class chaosGenerator :    
    def setLorenz (self, sol) :
        self.x0 = sol[-1,0]
        self.y0 = sol[-1,1]
        self.z0 = sol[-1,2]
    
    def __init__ (self, cmap) :
        
        self.cmap = cmap
        
        if cmap[0] == "logistic" or cmap[0] == "tent" :
            self.x0 = 0.01
            iterate = 1000 + int(np.random.rand() * 10000)
            funcMap = logisticMap if cmap[0] == "logistic" else tentMap
            for i in range (1, iterate) :
                self.x0 = funcMap (self.x0, cmap[1])
                
        elif cmap[0] == "lorenz" :
            self.x0 = np.random.rand ()
            self.y0 = np.random.rand ()
            self.z0 = np.random.rand ()
            sol = lorenzFlow ([self.x0, self.y0, self.z0], cmap[1], 1000, 100000)
            self.xmin = min(sol[:,0])
            self.xmax = max(sol[:,0])
            self.setLorenz (sol)
                
    def getTimeSeries (self, length) :
        if self.cmap[0] == "logistic" or self.cmap[0] == "tent" :
            n = np.arange (1, length + 1)
            xs = [self.x0]
            x = self.x0
            
            funcMap = logisticMap if cmap[0] == "logistic" else tentMap
            for i in range (1, length) :
                x = funcMap (x, self.cmap[1])
                xs.append (x)

            xs = np.array (xs)
            self.x0 = xs[-1]
            xs = np.reshape (xs, (-1, 1))
            
        elif self.cmap[0] == "lorenz" :
            sol = lorenzFlow ([self.x0, self.y0, self.z0], cmap[1], length, length * 100)
            inds = np.arange (0, length*100, 100).astype (int)
            xs = np.reshape (sol[:,0][inds], (-1, 1))
            xs = (xs - self.xmin)/(self.xmax - self.xmin)
            
            self.setLorenz (sol)
            
        return xs

    def getChaosPoints (self, shape) :
        prod = 1
        for i in range(0, len(shape)) :
            prod *= shape[i]
            
        return self.getTimeSeries(prod).reshape(shape)

# Initialise parameters

In [ ]:
D = 5
Np = 25
w = 0.1
c1 = 2
c2 = 2
beta = 0.9

# Initialise particles and velocities

In [ ]:
cmap = ["logistic", 4]
chaosMan = chaosGenerator (cmap)

particles = 20 * chaosMan.getChaosPoints ((Np, D)) - 10
velocity = chaosMan.getChaosPoints ((Np, D))
momentum = np.zeros (shape=(Np,D))
pbest = particles
obj = rastrigin

gbest = min (particles , key = lambda x : obj(x.reshape(1,-1))[0])

# Optimization loop

In [ ]:
for i in range (0, 500) :
    r1 = chaosMan.getChaosPoints ((Np, D))
    r2 = chaosMan.getChaosPoints ((Np, D))
    
    momentum = beta*momentum + (1-beta)*velocity
    velocity = momentum + c1*r1*(pbest - particles) + c2*r2*(gbest - particles)
    particles = particles + velocity
        
    less = obj(particles) < obj(pbest)
    pbest[less] = particles[less]
    gbest = min (pbest , key = lambda x : obj(x.reshape(1,-1))[0])
    
print ("f" + str(gbest) + " = " + str(obj(gbest.reshape(1,-1))[0]))